# Results to test different methods for approximating access costs at zone level

In [64]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

column_mapping = {'KNN_5': 'knn_5', 'KNN_15': 'knn_15', 'KNN_25': 'knn_25'}

In [43]:
# open a file, where you stored the pickled data
file = open('results/performance.pkl', 'rb')
# dump information to that file
performance = pickle.load(file)
# close the file
file.close()

# open a file, where you stored the pickled data
file = open('results/processing_times.pkl', 'rb')
# dump information to that file
processing_times = pickle.load(file)
# close the file
file.close()

In [70]:
res_list = []
it = 0

In [71]:
for stratum in ['wdam','wdpm','sat','bh']:
    iteration_results = performance[it][stratum]
    iteration_processing_times = processing_times[it][stratum]

    iteration_processing_times = iteration_processing_times.rename(columns=column_mapping)

    methods = list(iteration_results.columns[1:])


    for m in methods:
        res_append = {}
        res_append['stratum'] = stratum
        res_append['method'] = m
        res_append['corr'] = np.corrcoef(iteration_results[m],iteration_results['GM'])[0][1]
        res_append['cost'] = iteration_processing_times[m].sum()
        res_list.append(res_append)

results = pd.DataFrame(res_list)

In [76]:
results.to_csv('results_check.csv')

   stratum               method      corr          cost
0     wdam                knn_5 -0.554977   2867.533851
1     wdam               knn_15 -0.785025   9016.653957
2     wdam               knn_25 -0.884344  15465.664888
3     wdam              kmean_3  0.573496   1964.222930
4     wdam              kmean_5  0.581242   3277.224347
5     wdam              kmean_7  0.693475   4548.941149
6     wdam              kmean_9  0.687243   5981.403101
7     wdam        dbscan_0.01_1  0.810884  15823.453326
8     wdam        dbscan_0.01_3  0.816615  19494.872057
9     wdam        dbscan_0.01_5  0.898332  27846.875422
10    wdam        dbscan_0.02_1  0.238606   1409.029769
11    wdam        dbscan_0.02_3  0.238606   1409.029769
12    wdam        dbscan_0.02_5  0.229477   2467.158040
13    wdam        dbscan_0.03_1  0.233480    779.089080
14    wdam        dbscan_0.03_3  0.233480    779.089080
15    wdam        dbscan_0.03_5  0.233480    779.089080
16    wdam          hdbscan_3_1  0.593722   8600